### ایمپورت کردن کتابخانه ها

In [11]:
# !pip install requests
import requests
from bs4 import BeautifulSoup
import pandas as pd
from threading import Thread
from urls import urls

ModuleNotFoundError: No module named 'requests'

In [2]:
digikala_results = []
def scrape_digikala(url,list_digikala_results):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    results = soup.find("div",{"class_":"grow min-w-0"})
    results_name = results.find("h1",{"class_":"text-h4 text-neutral-900 mb-2 pointer-events-none"})
    name = results.text.strip()
    print(name)
    price_result = soup.find("span",{"class_":"text-h4 ml-1 text-neutral-800"})
    price = price_result.text.strip()
    list_digikala_results.append({"name":name,"price":price})

scrape_digikala("https://www.digikala.com/product/dkp-14664702/%D8%A7%D8%B3%D9%BE%D8%B1%D8%B3%D9%88-%D8%B3%D8%A7%D8%B2-%DA%AF%D9%88%D8%B3%D9%88%D9%86%DB%8C%DA%A9-%D9%85%D8%AF%D9%84-gem-873/",digikala_results)

NameError: name 'requests' is not defined

In [4]:
!pip install requests

### نوشتن تابع برای جمع آوری اطلاعات از سایت آمازون

In [ ]:
def scrape_amazon(keyword, result_list):
    url = f"https://www.amazon.com/s?k={keyword}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    for item in soup.select('.s-main-slot .s-result-item'):
        name = item.select_one('h2 a span')
        price = item.select_one('.a-price-whole')
        link = item.select_one('h2 a')

        if name and price and link:
            result_list.append({
                'نام محصول': name.get_text(),
                'قیمت در آمازون': float(price.get_text().replace(',', '')),
                'لینک به آمازون': "https://www.amazon.com" + link['href']
            })

### نوشتن تابع برای جمع آوری اطلاعات از سایت دیجی کالا

In [ ]:
def scrape_digikala(keyword, result_list):
    url = f"https://www.digikala.com/search/?q={keyword}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    for item in soup.select('.c-product-box'):
        name = item.select_one('.c-product-box__title')
        price = item.select_one('.c-price__value')
        link = item.select_one('.c-product-box__title a')

        if name and price and link:
            result_list.append({
                'نام محصول': name.get_text().strip(),
                'قیمت در دیجی‌کالا': float(price.get_text().replace(',', '')),
                'لینک به دیجی‌کالا': "https://www.digikala.com" + link['href']
            })

 ### نوشتن تابع برای جمع آوری اطلاعات از سایت اولفا

In [ ]:
def scrape_olfa(keyword, result_list):
    url = f"https://www.digikala.com/search/?q={keyword}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    for item in soup.select('.c-product-box'):
        name = item.select_one('.c-product-box__title')
        price = item.select_one('.c-price__value')
        link = item.select_one('.c-product-box__title a')

        if name and price and link:
            result_list.append({
                'نام محصول': name.get_text().strip(),
                'قیمت در اولفا': float(price.get_text().replace(',', '')),
                'لینک به اولفا': "https://www.olfacoffee.com" + link['href']
            })

### تعریف لیست ها برای ذخیره نتایج

In [ ]:
amazon_results = []
digikala_results = []
olfa_results = []

### تعریف کلمات کلیدی برای جست و جو

In [ ]:
keyword_amazon = "espresso machine"
keyword_digikala = "اسپرسو ساز"
keyword_olfa = ""

### ایجاد ترد ها

In [ ]:
thread_amazon = Thread(target=scrape_amazon, args=(keyword_amazon, amazon_results))
thread_digikala = Thread(target=scrape_digikala, args=(keyword_digikala, digikala_results))
thread_olfa = Thread(target=scrape_olfa, args=(keyword_olfa, olfa_results))

### شروع به کار ترد ها

In [ ]:
thread_amazon.start()
thread_digikala.start()
thread_olfa.start()

### منتظر ماندن برای اتمام ترد ها

In [ ]:
thread_amazon.join()
thread_digikala.join()
thread_olfa.join()

### تبدیل نتایج به دیتا فریم

In [ ]:
df_amazon = pd.DataFrame(amazon_results)
df_digikala = pd.DataFrame(digikala_results)
df_olfa = pd.DataFrame(olfa_results)

### تغییر واحد پولها به دلار  
بر فرض دلار 60,000 تومان

In [ ]:
exchange_rate = 60000
df_digikala['قیمت در دیجی کالا (دلار)'] = df_digikala['قیمت در دیجی‌کالا'] / exchange_rate
df_digikala['قیمت در اولفا (دلار)'] = df_olfa['قیمت در اولفا'] / exchange_rate
df_digikala['قیمت در آمازون (دلار)'] = df_amazon['قیمت در آمازون'] #the price is already by dollors .

### محاسبه ی کمترین و بیشترین قیمت محصولات

In [ ]:
min_prices = pd.concat([df_amazon[['نام محصول', 'قیمت در آمازون']], df_digikala[['نام محصول', 'قیمت در دیجی‌کالا (دلار)']].rename(columns={'قیمت در دیجی‌کالا (دلار)': 'قیمت'}), df_olfa[['نام محصول', 'قیمت در اولف (دلار)']].rename(columns={'قیمت در اولفا (دلار)': 'قیمت'})]).groupby('نام محصول').min().reset_index()
max_prices = pd.concat([df_amazon[['نام محصول', 'قیمت در آمازون']], df_digikala[['نام محصول', 'قیمت در دیجی‌کالا (دلار)']].rename(columns={'قیمت در دیجی‌کالا (دلار)': 'قیمت'}), df_olfa[['نام محصول', 'قیمت در اولف (دلار)']].rename(columns={'قیمت در اولفا (دلار)': 'قیمت'})]).groupby('نام محصول').max().reset_index()


### نمونه ی هزینه ها  
بر حسب دلار

#### هزینه خرید محصول

In [ ]:
cost_price = 180

#### هزینه حمل و نقل

In [ ]:
shipping_cost = 20

#### هزینه بازاریابی

In [ ]:
marketing_cost = 10

#### درصد سود موردنظر

In [ ]:
desired_profit_percentage = 0.20

### محاسبه قیمت پایه

In [ ]:
base_price = cost_price + shipping_cost + marketing_cost

### محاسبه قیمت فروش

In [ ]:
desired_profit = base_price * desired_profit_percentage
selling_price = base_price + desired_profit

### تعیین بازه قیمتی رقابتی

In [ ]:
min_competitor_price = min_prices['قیمت'].min()
max_competitor_price = max_prices['قیمت'].max()

### محصولات مشابه 

In [ ]:
# if there is subscriptions:
if not min_prices.empty:
    competitive_price_range = (base_price, min_competitor_price)
# if there isn't any subscriptions:
else:
    competitive_price_range = (base_price, base_price + desired_profit)

### نمایش نتایج

In [ ]:
print(f"قیمت پایه: {base_price}")

In [ ]:
print(f"قیمت فروش پیشنهادی: {selling_price}")

In [ ]:
print(f"بازه قیمتی رقابتی: از {competitive_price_range[0]} تا {competitive_price_range[1]}")

### نمایش قیمت رقبا برای هر محصول 

#### نمایش کمترین قیمت رقبا :

In [ ]:
print("\nکمترین قیمت رقبا برای هر محصول:")
print(min_prices)

#### نمایش بیشترین قیمت رقبا :

In [ ]:
print("\nبیشترین قیمت رقبا برای هر محصول:")
print(max_prices)

### ذخیره اطلاعات به صورت فایل csv

In [ ]:
df_amazon.to_csv('amazon_products.csv', index=False)
df_digikala.to_csv('digikala_products.csv', index=False)
df_olfa.to_csv('olfa_products.csv', index=False)